# Notebook for Capstone Week 3 Assignment

In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [6]:
#scraping the table into a dataframe using BeautifulSoup 
res = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.")

soup = BeautifulSoup(res.content,'lxml')

table = soup.find_all('table')[0]

df = pd.read_html(str(table))

In [7]:
TempDf = pd.DataFrame(df[0])

In [8]:
TempDf.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
#Eliminating rows in which Borough is not assigned
TempDf = TempDf[TempDf.Borough != 'Not assigned']

In [10]:
TempDf.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [11]:
#Replacing neighborhood name with borough name where neighborhood is not assigned
TempDf['Neighbourhood']=TempDf['Neighbourhood'].replace('Not assigned', TempDf['Borough'])

In [12]:
#Combining neighborhoods of the same borough into the same row
Toronto_df = TempDf.groupby(by=['Postcode','Borough']).agg(lambda x: ', '.join(x))

Toronto_df.reset_index(level=['Postcode','Borough'], inplace=True)

In [13]:
#Shape of the dataframe
Toronto_df.shape

(103, 3)

In [14]:
#Loading the table containing the coordinates of Toronto postcodes
path = 'https://cocl.us/Geospatial_data'
GeoData = pd.read_csv(path)
GeoData.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
#Merging the geodata dataframe with the Toronto postcodes dataframe
Toronto_df = Toronto_df.merge(GeoData, left_on='Postcode', right_on='Postal Code')
Toronto_df.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [16]:
#Droping the redundant column 'Postal Code'
Toronto_df.drop(['Postal Code'], axis=1, inplace=True)
pd.set_option('display.max_rows', 150)
Toronto_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
